In [62]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

from sklearn import linear_model

import scipy.io.wavfile as wavfile

In [63]:
#Using linear regression on the start of 50 audio samples to extrapolate the rest of the data.

#Percentage of audio clip kept.
Provided_Portion = 0.25

#Load in the 50 sample recordings from the Free-Spoken-Digits-Dataset (https://github.com/Jakobovski/free-spoken-digit-dataset/tree/master/recordings).
audio_files = [wavfile.read("data/free-spoken-digit-dataset-master/recordings/0_jackson_{}.wav".format(n))[1] for n in range(50)]
sample_rates = [wavfile.read("data/free-spoken-digit-dataset-master/recordings/0_jackson_{}.wav".format(n))[0] for n in range(50)]
audio_files = pd.DataFrame(audio_files, dtype=np.int16).dropna(axis=1).values

n_audio_samples = audio_files.shape[1]

In [64]:
model = linear_model.LinearRegression()

#Randomly select an audio file as the test data.
np.random.seed(17)
random_idx = np.random.randint(audio_files.shape[0])

test  = audio_files[random_idx]
sample_rate = sample_rates[random_idx]
train = np.delete(audio_files, [random_idx], axis=0)

In [65]:
train.shape
test.shape

(49, 4087)

(4087,)

In [66]:
wavfile.write('Original Test Clip.wav', sample_rate, test)

In [67]:
split = int(Provided_Portion*n_audio_samples)
X_test = test[:split].reshape(1, -1)
y_test = test[split:].reshape(1, -1)

X_train = train[:, :split]
y_train = train[:, split:]

model.fit(X_train, y_train)
y_test_prediction = model.predict(X_test).astype(dtype=np.int16)
y_test_prediction

#Note the low score below is not suprising. It had to extrapolate three times further than the training data. Nevertheless, the extrapolated audio is understandable.
score = model.score(X_test, y_test)
print("Extrapolation R^2 Score: ", score)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

array([[-3697, -2611,  -950, ...,   365,   -91,  -482]], dtype=int16)

Extrapolation R^2 Score:  0.0


In [68]:
completed_clip = np.hstack((X_test, y_test_prediction))
wavfile.write('Extrapolated Clip.wav', sample_rate, completed_clip[0])